# import das bibliotecas

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from glob import glob
from scipy import stats

rcParams['figure.figsize'] = [16., 5.]

### Filtros

In [2]:
# definições de filtros

def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)

def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)

def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

### Carregando Dados

In [3]:
# def plotData(dirs):
#     data = np.load("datasets/topicos_cc/"+dirs)
#     data = np.transpose(data, (0, 2, 1))
#     print(data.shape)
#     data_filtered = butter_notch(data, 60)
#     data_filtered = butter_highpass(data_filtered, 5)
#     data_filtered = butter_lowpass(data_filtered, 50)
#     for i in range(data_filtered.shape[1]):
#         plt.plot(data_filtered[0,i,:])
#     plt.suptitle(dirs)
#     plt.show()


In [4]:
def loadSujeito(dirs):
    data = np.load("datasets/topicos_cc/"+dirs)
    data = np.transpose(data, (0, 2, 1))
    data_filtered = butter_notch(data, 60)
    data_filtered = butter_highpass(data_filtered, 5)
    data_filtered = butter_lowpass(data_filtered, 50)
    
    return data_filtered

In [5]:
dirs = [ i.split("/")[-1] for i in glob('datasets/topicos_cc/p1*')]
data = []
for d in dirs:
    data.append(loadSujeito(d))

In [6]:
np.array(data).shape

(3, 8, 4, 1600)

### Visualizando

In [7]:
# for d in dirs:
#     plotData(d)

# Concatenando Trials x Movimentos

In [8]:
import numpy as np

#X = data.reshape(24, 4, 1600)
X = np.concatenate((data), axis=0)
X.shape


(24, 4, 1600)

## Segmentação dos dados

In [9]:
from scipy.signal import stft
data = X
step = 11.8
segment = 128
# data = X.reshape(24, 4, 1600)
# print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * int(step)

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=115)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')



Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (24, 4, 125, 128) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (24, 4, 125, 65) - (classes+ensaios, canais, janelas, linhas)


## Achar as Janelas
O mesmo que a função acima, mas generico

In [10]:
# from scipy.signal import stft

# def janela(overl=64):
    
#     for i in np.arange(1, 128, 0.1):
#         step = i
#         segment = 128
#         data = X.reshape(24, 4, 1600)
        
#         _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=overl)
#         chunks_freq = np.swapaxes(chunks_freq, 2, 3)
#         window = chunks_freq.shape[2] 

#         n_win = int((data.shape[-1] - segment) / step) + 1
#         ids = np.arange(n_win) * int(step)

#         # Janelas do dado no dominio do tempo
#         chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)
#         time_window = chunks_time.shape[2]
            
#         if( time_window == window ):
#             return step
        
    
# step = janela(overl=64)
# print(step)

In [11]:
# all_steps = []
# for i in [0.5, 0.7, 0.8, 0.9 ]:
#     n_step = int(128*i)
#     print(n_step)
#     step = janela(overl=n_step)
#     all_steps.append(step)
    
# all_steps

In [12]:
# from scipy.signal import stft

# def time_freq(step):
    
#     segment = 128
#     data = X.reshape(24, 4, 1600)
#     print('', data.shape)

#     n_win = int((data.shape[-1] - segment) / step) + 1
#     ids = np.arange(n_win) * int(step)

#     # Janelas do dado no dominio do tempo
#     chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

#     # Janelas do dado no domínio da frequência
#     _, _, chunks_freq = stft(data, fs=200, nperseg=128, noverlap=64)
#     chunks_freq = np.swapaxes(chunks_freq, 2, 3)

#     print('Formato (shape) dos dados depois da divisão de janelas')
#     print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
#     print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')
#     return chunks_time, chunks_freq

In [13]:
# chunks_time , chunks_freq = time_freq(11.8)

## Extração e seleção de características

In [14]:
def getZC(data, th):
    tamanho = len(data)
    somatoria = 0
    
    for i in range(tamanho-1):
        resultado = (data[i] * data[i+1] )
        resultado2 = np.abs(data[i] - data[i+1])
        if(resultado < 0 ) and (resultado2 > th):
            somatoria += 1
        
    return somatoria

def fj(i, sampleRate, tamanho):
    return i * sampleRate / (2 * tamanho)

def getFMN(data):
    tamanho = len(data)
    somatoria = 0
    sumPSD = np.sum(PSD(data))
    for i in range(tamanho):
        somatoria += (fj(i, 41, tamanho) * PSD(data[i]) ) / sumPSD
        
    return somatoria



In [15]:
from math import prod

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def wamp(x, th):
    res = np.abs(np.diff(x))
    return np.sum(res >= th, axis=-1)

def wl(x):
    res = np.abs(np.diff(x))
    return np.sum(res, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1])))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def zc(data,threshold):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                fy.append( getZC(data[xx][yy][zz], threshold ) )
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def fmn(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getFMN(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def A(w):
    return np.abs(w)

def getMMNF(data):
    tamanho = len(data)
    somatoria = 0
    
    sumA = np.sum(A(data))
    
    for i in range(tamanho):
        somatoria += (fj(i, 200, tamanho) * A(data[i]) ) / sumA 
        
    return somatoria

def mmnf(data):
    f =[]
    x,y,z = data.shape[:3]
    for xx in range(x):
        fx = []
        for yy in range(y):
            fy = []
            for zz in range(z):
                
                fy.append( getMMNF(data[xx][yy][zz]) )
                
            fx.append(fy)
        f.append(fx)
    return np.array(f)

def logD(data):
    from math import e
    N = np.prod(data.shape)
    
    return e ** ( np.sum(np.log10( np.abs(data) ), axis=-1) ) / N

def iemg(data):
    # tempo
    return np.sum(A(data), axis=-1)

def dasdv(data):
    #tempo
    return np.sqrt( np.sum(np.diff(data) ** 2, axis=-1) / (np.prod(data.shape[:-1]) - 1) )

def tmx(x, n):
    N = np.prod(x.shape[:-1])
    return np.abs(np.sum(x ** n, axis=-1) / N)

## Implementação do vetor

In [16]:
th = np.median(chunks_time)

In [17]:
#VAR, RMS, WL, TM5 e DASDV
#WL, IEMG, LOGD
final_data = list()
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(wamp(chunks_time, th))
final_data.append(logD(chunks_time))
final_data.append(wl(chunks_time))
final_data.append(zc(chunks_time,0))

final_data.append(iemg(chunks_time))
final_data.append(dasdv(chunks_time))
final_data.append(tmx(chunks_time, 3))
final_data.append(tmx(chunks_time, 4))
final_data.append(tmx(chunks_time, 5))

final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(fmn(chunks_freq))
final_data.append(mmnf(chunks_freq))

f, Pxx_den = signal.welch(data, fs=200, nperseg=248, noverlap=223)
final_data.append(Pxx_den)

final = np.array(final_data)
final.shape

(16, 24, 4, 125)

In [18]:
# data = final.transpose(0, 1, 3, 2)
# sh = data.shape

# X = data.reshape(sh[0], int(sh[1]/3), 3 * sh[2], sh[3])

# print(X.shape)

## PCA


In [19]:
from sklearn.decomposition import PCA

In [20]:
# pca = PCA(n_components=2)

# features = list()
# for f in X:
#     classes = list()
#     for c in f:
#         C_pca = pca.fit_transform(c)
#         classes.append(C_pca)
#     features.append(classes)

# X_pca = np.array(features)

In [21]:
# X_pca.shape

## Visualização

In [22]:
# def plot_features(features, features_names, classes_names, ch_1, ch_2):
    
#     movs = np.arange(len(classes_names))
#     markers = ["o", "v", "^", "P", "*", "x", "X", "2", "3", "1", 'm', 'L', 'z', 'U', '6']
#     for f, feature in enumerate(features):
        
#         for mov, marker in zip(movs, markers):
#             # argumentos: classes, amostras, canal
#             plt.scatter(feature[mov, :, ch_1],
#                         feature[mov, :, ch_2], marker=marker)

#         plt.legend((classes_names), scatterpoints=1, loc='best',
#                    ncol=3, fontsize=8)
        
#         plt.title(features_names[f])
#         plt.xlabel('CH{}'.format(ch_1))
#         plt.ylabel('CH{}'.format(ch_2))
#         plt.show()


In [23]:


# import matplotlib.pyplot as plt
# from matplotlib import rcParams

# plt.rcParams["figure.figsize"] = (12, 12)

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'iemg','dasdv','tm3','tm4','tm5', 'fmd', 'mmdf', 'fmn', 'mmnf')
# classes = [str(item) for item in list(range(8))]
# plot_features(X_pca, features_name, classes, 0, 1)


## Transpose para Selecionar Feature

In [24]:
final.shape
# 24*26 ,9, 4
# 24*26 , 10, 4
# 24*26 , 15, 4

(16, 24, 4, 125)

In [25]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(3000, 64)

In [26]:
# y = np.array(list(range(1, 9)) * int(X.shape[0] / 8)) # Antigo

y = [ [(i)] * int(X.shape[0] / 8 ) for i in range(8)]
y = np.array(y).flatten()
y.shape

(3000,)

## Seleção de características

## Variance Threshold


In [27]:
# # teste

# data_t = final.transpose(1, 3, 2, 0)
# ## X_t = data.reshape(24*26, 9, 4)
# ## X_t = data.reshape(24*26, 10, 4)
# ## X_t = data.reshape(24*26, 15, 4)
# X_t = data.reshape(24*26, 5, 4)

# data_t = X_t.transpose(2, 0, 1)
# data_t.shape


In [28]:
# from sklearn.feature_selection import VarianceThreshold
# canais = list()

# for c in data_t:
#     sel = VarianceThreshold(threshold=(.1))
#     vt = sel.fit_transform(c)
#     canais.append(vt)


### RFE (Por causa do Kernel Linear não iremos utilizar)

In [29]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVC
# estimator = SVC(kernel="linear")
# selector = RFE(estimator, n_features_to_select=5, step=1)
# selector = selector.fit(X, y)
# s = selector.fit_transform(X, y)

### GenericUnivariateSelect

In [30]:
# X.shape, y.shape

In [31]:
# from sklearn.feature_selection import GenericUnivariateSelect, chi2
# transformer = GenericUnivariateSelect(chi2, mode='k_best', param=10)
# X_new = transformer.fit_transform(X, y)
# X_new.shape

In [32]:
# from sklearn.feature_selection import SelectKBest
# X_new = SelectKBest(k=10).fit_transform(X, y)

In [33]:
# X.shape, X_new.shape

## Normalização

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_norm = scaler.fit_transform(X)

# X_new_norm = scaler.fit_transform(X_new)

## SVM

In [35]:
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

def do_svm(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)

    clf = SVC(gamma='scale')
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    
    acc = metrics.accuracy_score(y_test, y_pred)
    
    return acc

do_svm(X,y), do_svm(X_norm,y)

(0.11, 0.6266666666666667)

# Combinations

In [36]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         acc = do_svm(X_norm.take(res, axis=1), y)

#         if acc > best_acc:
#             best_acc = acc
#             best_comb = res

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}")

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'fmd', 'mmdf', 'fmn', 'mmnf')
# for i in best_comb:
#     print(f" {i} --- {features_name[i]}")


In [37]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         acc = do_svm(X.take(res, axis=1), y)

#         if acc > best_acc:
#             best_acc = acc
#             best_comb = res

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}")

# features_name = ('var', 'rms', 'wamp', 'wl', 'zc','logd', 'fmd', 'mmdf', 'fmn', 'mmnf')
# for i in best_comb:
#     print(f" {i} --- {features_name[i]}")


In [38]:
# from itertools import combinations

# best_acc = 0
# best_comb = 0
# best_k = 0
# for comb in range(1,12):
#     for res in combinations(range(9),comb):
#         for ks in range(1,41):
#             X_new = SelectKBest(k=ks).fit_transform(X, y)
#             acc = do_svm(X_new_norm.take(res, axis=1), y)

#             if acc > best_acc:
#                 best_acc = acc
#                 best_comb = res
#                 best_k = ks

# print(f"Melhor Acurácia: {best_acc}, Melhor Combinação: {best_comb}, Melhor K: {best_k}")




## RNN

In [39]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from urllib.request import urlopen, urlretrieve

import matplotlib.pyplot as plt
%matplotlib inline

2022-06-02 16:53:33.598175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 16:53:33.598212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Divisão dos dados em treino e teste:

In [40]:

# Dividindo em conjuntos de treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.3)

# treino: 80% dos 80% de treino. Validacao: 20% dos 80% de treino.
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.3, shuffle=True)


## Aplicação do algoritmo de MLP e geração dos resultados:

In [ ]:
# definição de uma fração do regularizador
l = 0.01

# desenvolvimento do modelo Keras para uma MLP
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=64,
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu',
                kernel_regularizer=regularizers.l2(l)))
# Aplicação de um dropout (caso necessário)
# model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# Aplicação de um modelo de descida de gradiente utilizando o Stocastic Gradient Descendent (SGD)
sgd = SGD(lr=0.05, momentum=0.0)
# Função de otimização da rede: ADAM
adam = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# Função de custo baseada em dados originalmente categóricos
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam,
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=550, batch_size=15,
                    validation_data=(X_val, y_val))

Epoch 1/550


2022-06-02 16:53:36.032984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-02 16:53:36.033044: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-02 16:53:36.033077: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-02 16:53:36.033477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/alunos/a1858351/Documentos/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserW

98/98 [==============================] - 1s 6ms/step - loss: 2.1663 - accuracy: 0.2565 - val_loss: 1.9208 - val_accuracy: 0.3317
Epoch 2/550
98/98 [==============================] - 0s 3ms/step - loss: 1.7929 - accuracy: 0.3524 - val_loss: 1.7206 - val_accuracy: 0.4048
Epoch 3/550
98/98 [==============================] - 0s 4ms/step - loss: 1.6657 - accuracy: 0.4388 - val_loss: 1.6085 - val_accuracy: 0.4619
Epoch 4/550
98/98 [==============================] - 0s 4ms/step - loss: 1.5901 - accuracy: 0.4646 - val_loss: 1.5737 - val_accuracy: 0.4714
Epoch 5/550
98/98 [==============================] - 0s 4ms/step - loss: 1.5425 - accuracy: 0.4830 - val_loss: 1.5470 - val_accuracy: 0.4841
Epoch 6/550
98/98 [==============================] - 0s 4ms/step - loss: 1.4995 - accuracy: 0.5014 - val_loss: 1.5142 - val_accuracy: 0.5032
Epoch 7/550
98/98 [==============================] - 0s 4ms/step - loss: 1.4657 - accuracy: 0.5388 - val_loss: 1.4655 - val_accuracy: 0.5238
Epoch 8/550
98/98 [======

Epoch 59/550
98/98 [==============================] - 0s 3ms/step - loss: 0.9592 - accuracy: 0.7762 - val_loss: 1.0459 - val_accuracy: 0.7397
Epoch 60/550
98/98 [==============================] - 0s 4ms/step - loss: 0.9364 - accuracy: 0.7769 - val_loss: 1.1055 - val_accuracy: 0.6746
Epoch 61/550
98/98 [==============================] - 0s 3ms/step - loss: 0.9471 - accuracy: 0.7728 - val_loss: 1.2145 - val_accuracy: 0.6746
Epoch 62/550
98/98 [==============================] - 0s 3ms/step - loss: 0.9460 - accuracy: 0.7728 - val_loss: 1.1092 - val_accuracy: 0.7222
Epoch 63/550
98/98 [==============================] - 0s 4ms/step - loss: 0.9243 - accuracy: 0.7803 - val_loss: 1.0729 - val_accuracy: 0.7190
Epoch 64/550
98/98 [==============================] - 0s 3ms/step - loss: 0.9161 - accuracy: 0.7762 - val_loss: 1.0655 - val_accuracy: 0.7286
Epoch 65/550
98/98 [==============================] - 0s 3ms/step - loss: 0.9646 - accuracy: 0.7728 - val_loss: 1.0437 - val_accuracy: 0.7381
Epoch 

98/98 [==============================] - 0s 3ms/step - loss: 0.7995 - accuracy: 0.8177 - val_loss: 1.0292 - val_accuracy: 0.7286
Epoch 117/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7980 - accuracy: 0.8272 - val_loss: 0.9778 - val_accuracy: 0.7762
Epoch 118/550
98/98 [==============================] - 0s 4ms/step - loss: 0.8227 - accuracy: 0.8102 - val_loss: 1.0345 - val_accuracy: 0.7571
Epoch 119/550
98/98 [==============================] - 0s 4ms/step - loss: 0.8185 - accuracy: 0.8170 - val_loss: 1.0330 - val_accuracy: 0.7254
Epoch 120/550
98/98 [==============================] - 0s 4ms/step - loss: 0.8424 - accuracy: 0.7973 - val_loss: 1.0905 - val_accuracy: 0.7254
Epoch 121/550
98/98 [==============================] - 0s 3ms/step - loss: 0.8486 - accuracy: 0.8054 - val_loss: 1.0248 - val_accuracy: 0.7222
Epoch 122/550
98/98 [==============================] - 0s 4ms/step - loss: 0.8495 - accuracy: 0.8034 - val_loss: 1.0297 - val_accuracy: 0.7349
Epoch 123/550

98/98 [==============================] - 0s 4ms/step - loss: 0.7661 - accuracy: 0.8497 - val_loss: 1.0524 - val_accuracy: 0.7349
Epoch 174/550
98/98 [==============================] - 0s 3ms/step - loss: 0.8089 - accuracy: 0.8109 - val_loss: 1.0288 - val_accuracy: 0.7587
Epoch 175/550
98/98 [==============================] - 0s 4ms/step - loss: 0.8236 - accuracy: 0.8129 - val_loss: 1.0177 - val_accuracy: 0.7540
Epoch 176/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7858 - accuracy: 0.8422 - val_loss: 0.9547 - val_accuracy: 0.7873
Epoch 177/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7543 - accuracy: 0.8469 - val_loss: 1.0106 - val_accuracy: 0.7476
Epoch 178/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7843 - accuracy: 0.8279 - val_loss: 1.0079 - val_accuracy: 0.7302
Epoch 179/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7630 - accuracy: 0.8510 - val_loss: 0.9582 - val_accuracy: 0.7667
Epoch 180/550

98/98 [==============================] - 0s 3ms/step - loss: 0.7017 - accuracy: 0.8619 - val_loss: 1.0026 - val_accuracy: 0.7825
Epoch 231/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7168 - accuracy: 0.8524 - val_loss: 1.1214 - val_accuracy: 0.7111
Epoch 232/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7439 - accuracy: 0.8374 - val_loss: 0.9689 - val_accuracy: 0.7841
Epoch 233/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7405 - accuracy: 0.8490 - val_loss: 0.9740 - val_accuracy: 0.7603
Epoch 234/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7440 - accuracy: 0.8435 - val_loss: 1.1223 - val_accuracy: 0.7016
Epoch 235/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7973 - accuracy: 0.8279 - val_loss: 1.1098 - val_accuracy: 0.7206
Epoch 236/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7304 - accuracy: 0.8463 - val_loss: 0.9650 - val_accuracy: 0.7778
Epoch 237/550

98/98 [==============================] - 0s 4ms/step - loss: 0.7296 - accuracy: 0.8531 - val_loss: 1.0447 - val_accuracy: 0.7397
Epoch 288/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7295 - accuracy: 0.8497 - val_loss: 1.0341 - val_accuracy: 0.7667
Epoch 289/550
98/98 [==============================] - 0s 4ms/step - loss: 0.6949 - accuracy: 0.8667 - val_loss: 0.9839 - val_accuracy: 0.7683
Epoch 290/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7058 - accuracy: 0.8660 - val_loss: 1.0146 - val_accuracy: 0.7556
Epoch 291/550
98/98 [==============================] - 0s 4ms/step - loss: 0.6820 - accuracy: 0.8667 - val_loss: 1.0251 - val_accuracy: 0.7651
Epoch 292/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7369 - accuracy: 0.8503 - val_loss: 1.2970 - val_accuracy: 0.6952
Epoch 293/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7759 - accuracy: 0.8381 - val_loss: 1.0875 - val_accuracy: 0.7365
Epoch 294/550

98/98 [==============================] - 0s 3ms/step - loss: 0.7487 - accuracy: 0.8272 - val_loss: 1.0694 - val_accuracy: 0.7317
Epoch 345/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7649 - accuracy: 0.8333 - val_loss: 1.0674 - val_accuracy: 0.7508
Epoch 346/550
98/98 [==============================] - 0s 3ms/step - loss: 0.6953 - accuracy: 0.8592 - val_loss: 0.9867 - val_accuracy: 0.7587
Epoch 347/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7559 - accuracy: 0.8320 - val_loss: 1.1048 - val_accuracy: 0.7032
Epoch 348/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7553 - accuracy: 0.8435 - val_loss: 1.0588 - val_accuracy: 0.7651
Epoch 349/550
98/98 [==============================] - 0s 4ms/step - loss: 0.6901 - accuracy: 0.8721 - val_loss: 1.0666 - val_accuracy: 0.7429
Epoch 350/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7032 - accuracy: 0.8667 - val_loss: 1.0753 - val_accuracy: 0.7476
Epoch 351/550

98/98 [==============================] - 0s 4ms/step - loss: 0.6973 - accuracy: 0.8646 - val_loss: 0.9766 - val_accuracy: 0.7667
Epoch 402/550
98/98 [==============================] - 0s 3ms/step - loss: 0.7024 - accuracy: 0.8544 - val_loss: 1.1276 - val_accuracy: 0.7397
Epoch 403/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7323 - accuracy: 0.8490 - val_loss: 0.9782 - val_accuracy: 0.7683
Epoch 404/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7327 - accuracy: 0.8422 - val_loss: 1.0953 - val_accuracy: 0.7302
Epoch 405/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7136 - accuracy: 0.8510 - val_loss: 0.9234 - val_accuracy: 0.7889
Epoch 406/550
98/98 [==============================] - 0s 4ms/step - loss: 0.6602 - accuracy: 0.8714 - val_loss: 0.9534 - val_accuracy: 0.7730
Epoch 407/550
98/98 [==============================] - 0s 4ms/step - loss: 0.7346 - accuracy: 0.8422 - val_loss: 1.2155 - val_accuracy: 0.7190
Epoch 408/550

In [ ]:
# 
# score = model.predict_classes(X_test)
# y_true = [np.where(x == 1)[0][0] for x in y_test]

predict_x=model.predict(X_test) 
score=np.argmax(predict_x,axis=1)
y_true = y_test

print('Acurácia: %0.2f%%' % (accuracy_score(y_true, score) * 100))
print('Matriz de confusão:')
print(confusion_matrix(y_true, score))
print()
print(classification_report(y_true, score, digits=5))

# Grafico

In [ ]:
def plot_history(h):
    loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in h.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in h.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Custo não está presente no histórico')
        return
    epochs = range(1, len(history.history[loss_list[0]]) + 1)
    # Custo
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Custo [treinamento] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Custo [validação] (' + str(str(format(
                    h.history[l][-1],'.5f'))+')'))
    plt.title('Custo')
    plt.xlabel('Épocas')
    plt.ylabel('Custo')
    plt.legend()
    # Acurácia
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, h.history[l], 'b',
                 label='Acurácia [treinamento] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    for l in val_acc_list:
        plt.plot(epochs, h.history[l], 'g',
                 label='Acurácia [validação] (' + str(format(
                    h.history[l][-1],'.5f'))+')')
    plt.title('Acurácia')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)